### Prerequisites

To run this .NET Interactive run book, you need a create an AppRegistration in Azure AD with the following settings
- Authentication -> Platform: Mobile and Desktop
- Authentication -> Platform -> RedirectUris: Msal...
- Authentication -> Supported Account Types: Single Tenant
- Authentication -> Advanced Settings -> Allow public client flows: True
- Press **Save**

- API Permissions -> Add a Permission -> Delegated Permisions -> MS Graph: `Application.ReadWrite.All`

We start by importing the necessary NuGet packages to be used throughout this runbook

In [ ]:
#r "nuget:Microsoft.Graph,3.32.0"
#r "nuget:Microsoft.Identity.Client,4.31.0"
#r "nuget:Microsoft.Graph.Auth,1.0.0-preview.7"

Installed Packages Microsoft.Graph, 3.32.0 Microsoft.Graph.Auth, 1.0.0-preview.7 microsoft.graph.core, 1.25.1 Microsoft.Identity.Client, 4.31.0 newtonsoft.json, 9.0.1

### Create the MSAL Client
Ensure that you add the appropriate `usings` so that we can interact with Graph and AAD.

Instantiate the MSAL Client to authenticate against AAD and get the right Graph Permissions.
The MSAL Client requires the 2 following properties:
- `ClientID` (From the Overview tab of your AAD App Registration)
- `TenantID` (From the Overview tab. Can be the tenant id [Guid] or the tenant name)

In [ ]:
using Microsoft.Identity.Client;
using System.Threading.Tasks;
using Microsoft.Graph.Auth;

public static DeviceCodeProvider GetMsalClient()
{
    var clientId = "c369fdb7-d70b-4651-9e5c-4379c3863d78";
    var tenantId = "b55f0c51-61a7-45c3-84df-33569b247796";
    
    IPublicClientApplication publicClientApplication = PublicClientApplicationBuilder
        .Create(clientId)
        .WithTenantId(tenantId)
        .Build();
    var scopes = new string[]{"Application.ReadWrite.All", "User.Read"};
    return new DeviceCodeProvider(publicClientApplication, scopes);
}

### Create an API App Registration in Azure AD

Right now the scope is hardcoded to `api.read`. Feel free to change this to meet your needs

In [ ]:
using Microsoft.Graph;

var apiAppName = "Swagger Demo API";

var graphClient = new GraphServiceClient(GetMsalClient());
var apiApp = GetApiApplication(apiAppName);

var apiAppRegistration = await graphClient.Applications
                            .Request()
                            .AddAsync(apiApp);

private async Task<string> GetTenantId()
{
    var organization = await graphClient.Organization
                                .Request()
                                .GetAsync();
    return organization.FirstOrDefault().Id;
}

Console.WriteLine($"Client Id: {apiAppRegistration.AppId}");
Console.WriteLine($"Domain: {apiAppRegistration.PublisherDomain}");
Console.WriteLine($"Tenant Id: {await GetTenantId()}");

private static Application GetApiApplication(string displayName) => new Application
{
    DisplayName = displayName,
    IdentifierUris = new List<String>()
    {
        $"api://{Guid.NewGuid().ToString()}"
    },
    Api = new ApiApplication
    {
        AcceptMappedClaims = null,
        KnownClientApplications = new List<Guid>()
        {
        },
        RequestedAccessTokenVersion = 2,
        Oauth2PermissionScopes = new List<PermissionScope>()
        {
            new PermissionScope
            {
                Id = Guid.NewGuid(),
                AdminConsentDescription = "access the api as a reader",
                AdminConsentDisplayName = "access the api as a reader",
                IsEnabled = true,
                Type = "User",
                UserConsentDescription = "access the api as a reader",
                UserConsentDisplayName = "access the api as a reader",
                Value = "api.read"
            }
        },
        PreAuthorizedApplications = new List<PreAuthorizedApplication>()
        {
        }
    }
};

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F6URU5X4H to authenticate.


Client Id: c0b047a9-f3b2-424e-a22d-9379e85ec9cd


Domain: cmatskas.onmicrosoft.com


Tenant Id: b55f0c51-61a7-45c3-84df-33569b247796


### Create the Service Principal
Next we need to create a Service Principal that maps to the App Registration. 
This registers the app with the current Tenant and make the App Registration available to 
the tenant's apps and users to authenticate against and/or acquire tokens.

In [ ]:
using Microsoft.Graph;

var servicePrincipal = await graphClient.ServicePrincipals
                            .Request()
                            .AddAsync(CreateServicePrincipal(apiAppRegistration.AppId));

private static ServicePrincipal CreateServicePrincipal(string appId) => new ServicePrincipal
{
    AppId = appId
};

### App Registration
Create a client App Registration for a server-side **web app** (.NET, Node, Java etc).

Users signing in to the client app, will be able to request an access token for the API.
You'll need to set the following two properties before running this script:
- `clientAppName` -> the name for your App Registration
- `redirectUri` -> the URI where your app expects the returned tokens. For OpenAPI the expected Redirect URI 
 is `https://localhost:5001/swagger/oauth2-redirect.html`

In [ ]:
using Microsoft.Graph;

var clientAppName = "Swagger Demo Client";
var redirectUri = "https://localhost:5001/swagger/oauth2-redirect.html";

var clientApp = GetClientApplication(
    apiAppRegistration,
    clientAppName,
    redirectUri);

var clientAppRegistration = await graphClient.Applications
                                .Request()
                                .AddAsync(clientApp);

Console.WriteLine($"Client Id: {clientAppRegistration.AppId}");
private static Application GetClientApplication(Application app,string displayName, string redirectUri) 
    => new Application
{
    DisplayName = displayName,
    Spa = new SpaApplication
    {
        RedirectUris = new List<string>(){redirectUri}
    },
    RequiredResourceAccess = new List<RequiredResourceAccess>()
    {
        new RequiredResourceAccess
        {
            ResourceAppId = app.AppId,
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {
                    Id = app.Api.Oauth2PermissionScopes.FirstOrDefault().Id,
                    Type = "Scope"
                }
            }
        },
        new RequiredResourceAccess
        {
            // OIDC - Graph scope (user.read)
            ResourceAppId = "00000003-0000-0000-c000-000000000000",
            ResourceAccess = new List<ResourceAccess>()
            {
                new ResourceAccess
                {     
                    Id = new Guid("e1fe6dd8ba314d6189e788639da4683d"),
                    Type = "Scope"
                }
            }
        }
    } 
};

Client Id: 3af475f5-94a3-4d12-a581-4b8b47064fd9
